In [1]:
!pip install -U ell-ai
!pip install -U openai

    torch (>=1.7torchvision)
          ~~~~~~^
    torch (>=1.7torchvision)
          ~~~~~~^
  Attempting uninstall: openai
    Found existing installation: openai 1.50.0
    Uninstalling openai-1.50.0:
      Successfully uninstalled openai-1.50.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aider-chat 0.50.1 requires numpy==1.26.4, but you have numpy 2.1.1 which is incompatible.
aider-chat 0.50.1 requires openai==1.40.6, but you have openai 1.50.1 which is incompatible.
graphrag 0.1.1 requires numpy<2.0.0,>=1.25.2, but you have numpy 2.1.1 which is incompatible.
graphrag 0.1.1 requires scipy==1.12.0, but you have scipy 1.13.1 which is incompatible.


In [2]:
# OpenAI version
from openai import OpenAI
from IPython.display import Markdown, display
import os

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)

system_prompt = """
You are tasked to review pull requests, focusing on identifying logic errors, potential bugs in edge cases, security vulnerabilities, and ensuring code clarity, while avoiding opinionated feedback on coding style or minor issues.

Here are concise instructions for reviewing PRs:
1. Focus on:
   - Logic errors
   - Edge case handling
   - Security vulnerabilities
   - Code clarity and maintainability
2. Suggest changes only for:
   - Logical errors
   - Potential bugs
   - Security issues
   - Significant performance improvements
   - Unclear variable/function names
3. Avoid:
   - Opinionated comments on coding style
   - Enforcing personal preferences
   - Nitpicking minor issues
4. Ask questions to understand the code's purpose and context if unclear.
5. Provide clear explanations for any suggested changes.
6. Be constructive and respectful in all comments.
"""

user_prompt = """
<<<<<<< ORIGINAL
    String.match?(param, ~r/^asset:[a-z,0-9,-]{26}$/)
=======
    String.match?(param, ~r/^((ab:oop:cts:ie:)?asset:[a-z,0-9,-]{26})$/) or
      String.match?(param, ~r/^ac:opp:cs:asset:[a-z,0-9]{12}$/) or
      String.match?(param, ~r/^((ac:)?(opp:)?pips:(id:)?[a-z,0-9]{8})$/)
>>>>>>> UPDATED
"""
# user_prompt_context = f"""You are tasked to review a pull request that modifies regular expression patterns for asset identification.
# Analyze the following code diff, focusing on logic errors, potential bugs, security vulnerabilities, and code clarity.
# Provide either approval using the phrase "Good job!" or specific recommendations for changes, optionally with code examples.
# Be concise and direct in your feedback.

# {user_prompt}"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

display(Markdown(completion.choices[0].message.content))

The code appears to be matching a string `param` against several regular expression patterns. Here’s a review of the changes made in the pull request:

### Potential Issues Identified:

1. **Capture Group Misalignment**:
   - The regex patterns in the updated code contain additional capture groups (the `()`). While this may not be a direct error, if the intention is only to validate the format, capturing groups may not be needed as they could introduce complexity and confusion. If capturing groups are unnecessary for further processing, it's advisable to remove them for clarity.

2. **Comma Usage in Character Classes**:
   - In the regex, `~r/^asset:[a-z,0-9,-]{26}$/`, the commas appear in the character class `[a-z,0-9,-]` but do not serve a purpose in the ranges defined. The correct usage should remove the comma after `a-z`, as the character class should ideally just include ranges. It could simply be `[a-z0-9-]`.

3. **Possible Regex Overlap**:
   - The new update introduces three different regex patterns. Care should be taken to ensure that these patterns do not unintentionally overlap, leading to ambiguous matches. It would be beneficial to provide a clear distinction or condition under which each regex is meant to be used.

4. **Inconsistency in Length Expectations**:
   - The regex patterns validate different lengths for the asset strings (e.g., 26 characters for the first regex and 12 for the second), but there is no explanation of these specific lengths. Would it be beneficial for the patterns to have better documentation or comments explaining why those lengths are in place?

### Suggested Changes:

- **Remove Unnecessary Capture Groups**:
   If capture groups are not used later in the code, consider removing them to simplify the regex patterns.
   
- **Adjust Character Classes**:
   Revise the character classes in the regex to remove unnecessary commas:
   ```elixir
   String.match?(param, ~r/^((ab:oop:cts:ie:)?asset:[a-z0-9-]{26})$/) or
      String.match?(param, ~r/^ac:opp:cs:asset:[a-z0-9]{12}$/) or
      String.match?(param, ~r/^((ac:)?(opp:)?pips:(id:)?[a-z0-9]{8})$/)
   ```

- **Add Comments for Clarity**:
   Consider adding comments above the regex patterns to describe what each pattern is intended to match. This would improve maintainability and help other developers understand the code's purpose quickly.

### Request for Clarification:
Could you clarify if there's a specific use case or requirement that mandates the capture groups and the specific lengths of the asset identifiers? Understanding the rationale could help ensure the patterns remain robust against future changes.

In [3]:
from IPython.display import Markdown, display
import ell

@ell.simple(model="gpt-4o-mini")
def review(change: str):
    """You are tasked to review pull requests, focusing on identifying logic errors, potential bugs in edge cases, security vulnerabilities, and ensuring code clarity, while avoiding opinionated feedback on coding style or minor issues.

Here are concise instructions for reviewing PRs:
1. Focus on:
   - Logic errors
   - Edge case handling
   - Security vulnerabilities
   - Code clarity and maintainability
2. Suggest changes only for:
   - Logical errors
   - Potential bugs
   - Security issues
   - Significant performance improvements
   - Unclear variable/function names
3. Avoid:
   - Opinionated comments on coding style
   - Enforcing personal preferences
   - Nitpicking minor issues
4. Ask questions to understand the code's purpose and context if unclear.
5. Provide clear explanations for any suggested changes.
6. Be constructive and respectful in all comments."""
    return f"{change}"

pr_change = """
<<<<<<< ORIGINAL
    String.match?(param, ~r/^asset:[a-z,0-9,-]{26}$/)
=======
    String.match?(param, ~r/^((ab:oop:cts:ie:)?asset:[a-z,0-9,-]{26})$/) or
      String.match?(param, ~r/^ac:opp:cs:asset:[a-z,0-9]{12}$/) or
      String.match?(param, ~r/^((ac:)?(opp:)?pips:(id:)?[a-z,0-9]{8})$/)
>>>>>>> UPDATED
"""

display(Markdown(review(pr_change)))

The updated code modifies the matching logic for the `param` variable. Here are some points to consider regarding potential logic errors or improvements:

1. **Regex Overlap**:
   - The original regex only matches strings that start with `asset:` followed by 26 characters (lowercase letters, numbers, and dashes). 
   - The updated regex includes multiple patterns, but they may not be mutually exclusive. This can lead to confusion regarding what patterns are being permitted and could lead to unexpected matches. Ensure that the intent of all included patterns is clear.

2. **Complexity and Readability**:
   - The updated regex logic is now more complex. This can decrease the clarity of the code. Consider breaking down these regex patterns into named functions or variables that explain their purpose. For example, you could use descriptive variables like `asset_regex`, `ac_opp_regex`, etc.

3. **Performance**:
   - Depending on how frequently this matching is performed and the length of `param`, the additional checks could impact performance. Consider evaluating if a single regex pattern could represent all the desired cases, as regex matching can be computationally expensive.

4. **Edge Case Handling**:
   - The use of `or` for separate regex patterns means they will be evaluated independently. Ensure that there are tests covering edge cases for each regex pattern to validate their logic. For example, check if inputs that closely resemble the regex but don't meet all requirements are handled appropriately.

5. **Potential Bugs**:
   - Ensure that your regex patterns do not inadvertently allow other formats that should be restricted. Each pattern should be thoroughly tested:
     - For `asset:` prefix: ensure only valid 26-character combinations are permitted.
     - For `ac:opp:cs:asset:` prefix: confirm it accepts only valid 12-character combinations.
     - For `ac:opp:pips:` check, ensure it strictly allows 8 characters without exceptions.

For improvement, you could restructure the implementation like this:

```elixir
def is_valid_asset(param) do
  asset_regex = ~r/^asset:[a-z,0-9,-]{26}$/
  acopp_cs_regex = ~r/^ac:opp:cs:asset:[a-z,0-9]{12}$/
  acopp_pips_regex = ~r/^((ac:)?(opp:)?pips:(id:)?[a-z,0-9]{8})$/

  String.match?(param, asset_regex) or
  String.match?(param, acopp_cs_regex) or
  String.match?(param, acopp_pips_regex)
end
```

This way, you encapsulate the regex patterns into a clearer structure that enhances readability while retaining functionality.

In [4]:
from IPython.display import Markdown, display
import ell

ell.init(verbose=True)

@ell.simple(model="gpt-4o-mini")
def review(change: str):
    return [
        ell.system("""You are tasked to review pull requests, focusing on identifying logic errors, potential bugs in edge cases, security vulnerabilities, and ensuring code clarity, while avoiding opinionated feedback on coding style or minor issues.

Here are concise instructions for reviewing PRs:
1. Focus on:
   - Logic errors
   - Edge case handling
   - Security vulnerabilities
   - Code clarity and maintainability
2. Suggest changes only for:
   - Logical errors
   - Potential bugs
   - Security issues
   - Significant performance improvements
   - Unclear variable/function names
3. Avoid:
   - Opinionated comments on coding style
   - Enforcing personal preferences
   - Nitpicking minor issues
4. Ask questions to understand the code's purpose and context if unclear.
5. Provide clear explanations for any suggested changes.
6. Be constructive and respectful in all comments."""),
        ell.user(f"{change}")
        # ell.assistant(...)
    ]

pr_change = """
<<<<<<< ORIGINAL
    String.match?(param, ~r/^asset:[a-z,0-9,-]{26}$/)
=======
    String.match?(param, ~r/^((ab:oop:cts:ie:)?asset:[a-z,0-9,-]{26})$/) or
      String.match?(param, ~r/^ac:opp:cs:asset:[a-z,0-9]{12}$/) or
      String.match?(param, ~r/^((ac:)?(opp:)?pips:(id:)?[a-z,0-9]{8})$/)
>>>>>>> UPDATED
"""

display(Markdown(review(pr_change)))

╔══════════════════════════════════════════════════════════════════════════════╗
║ review(
<<<<<<<..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│      system: You are tasked to review pull requests, focusing on
│              identifying logic errors, potential bugs in edge cases,
│              security vulnerabilities, and ensuring code clarity, while
│              avoiding opinionated feedback on coding style or minor
│              issues.
│              Here are concise instructions for reviewing PRs:
│              1. Focus on:
│                 - Logic errors
│                 - Edge case handling
│                 - Security vulnerabilities
│                 - Code clarity and maintainability
│              2. Suggest changes only for:
│                 - Logical errors
│                 - Potential bugs
│                 - Security issues
│        

In the provided code, it appears that the `match?` function is being used to validate a `param` against various regular expression patterns. Here are some considerations I would suggest regarding potential logic errors, edge case handling, and overall clarity:

1. **Logic Errors and Clarity:**
   - The use of multiple regex patterns combined with logical OR can potentially lead to unintended matches. For example, without clear separation or additional constraints, certain strings may match multiple patterns, making it unclear which type of asset is being validated. It is important to ensure that the regex patterns are mutually exclusive or properly documented.
   - Consider adding comments above the regex patterns to clarify what each pattern is expected to match. This would enhance code maintainability and help future developers understand the intent behind each regex.

2. **Edge Cases Handling:**
   - The first pattern `~r/^asset:[a-z,0-9,-]{26}$/` includes commas `,` which is incorrect in a character class for a lowercase letter or digit. It should just be `[a-z0-9-]{26}` without the comma.
   - The last regex pattern in the updated version, `~r/^((ac:)?(opp:)?pips:(id:)?[a-z,0-9]{8})$/`, also allows a comma in the character class, which seems unnecessary and could potentially cause issues. It should be adjusted to `[a-z0-9]{8}`.
   - Consider potential length discrepancies across different patterns that may lead to false validation. For example, ensure that lengths for valid strings are well-defined and that constraints are consistent across all patterns.

3. **Security Vulnerabilities:**
   - Ensure that the regex patterns do not allow for injection or misuse via unexpected inputs, especially if these patterns are exposed through APIs or used in any form of data input that may reach a database or external system.
   - Validate that the strings matched by these patterns do not lead to further actions in the system that can be exploited, such as processing or storing in ways that may allow malformed or malicious data.

4. **Performance Considerations:**
   - If this matching runs frequently and performance becomes a concern, consider how the regex patterns are compiled and matched. You may want to profile them to ensure they perform adequately under load.

Here’s a suggested version that also addresses the aforementioned points:

```elixir
# Validate 'param' against specified asset patterns.
String.match?(param, ~r/^asset:[a-z0-9-]{26}$/) or
  String.match?(param, ~r/^ac:opp:cs:asset:[a-z0-9]{12}$/) or
  String.match?(param, ~r/^((ac:)?(opp:)?pips:(id:)?[a-z0-9]{8})$/)
```

By modifying the character classes to eliminate commas, we enhance the clarity and correctness of the regex patterns. Consider adding inline comments to better document the purpose of each regex pattern.

In [5]:
# ollama example
from IPython.display import HTML, display
import ell
import json

# MODEL = "moondream:latest"
# client = OpenAI(
# 	base_url = "http://localhost:11434/v1",
# 	api_key = "ollama",
# )

ell.init(verbose=False)

ell.models.ollama.register(base_url="http://localhost:11434/v1")


@ell.simple(model="llama3.2:3b-instruct-q8_0")
def make_javascript_game(game_description: str):
    """You are an expert 2D browser game developer.
    You create simple, engaging games using vanilla JavaScript and HTML5 Canvas, minimizing external dependencies.
    You prioritize fun gameplay and broad accessibility over complex graphics.
    Your games are entirely self-contained in a single HTML file, ready to play by saving as index.html and opening in any browser.
    Always provide the complete game code within a single ```html``` code block."""
    return f"""Based on the following game description, create a complete, playable 2D browser game in a single HTML file, including all necessary HTML, CSS, and JavaScript. Provide the entire game code within a single ```html``` code block:
    {game_description}
    """

# Example game description
game_description = """Create a simple 2D platformer where a square character can jump between platforms to collect coins.
The game ends when all coins are collected or the player falls off the platforms."""

try:
    game = make_javascript_game(game_description)
    print("LLM Response:")
    print(game)  # Print the raw LLM output

    def display_game(game_html):
        try:
            html_content = game_html.split('```html')[1].split('```')[0].strip()
            
            iframe_html = f"""
            <iframe id="gameFrame" style="width: 100%; height: 400px; border: none;"></iframe>
            <button onclick="playGame()">Play Game</button>
            <script>
            function playGame() {{
                var gameFrame = document.getElementById('gameFrame');
                var gameContent = {json.dumps(html_content)};
                gameFrame.srcdoc = gameContent;
            }}
            </script>
            """
            display(HTML(iframe_html))
        except IndexError:
            print("Error: Could not extract HTML content. LLM response format unexpected.")
            print("Raw LLM output:")
            print(game_html)

    display_game(game)

except Exception as e:
    print(f"An error occurred: {str(e)}")

LLM Response:
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Platformer Game</title>
    <style>
        body {
            background-color: #f2f2f2;
            font-family: Arial, sans-serif;
        }
        
        canvas {
            border: 1px solid black;
        }

        #score-display {
            font-size: 24px;
            font-weight: bold;
        }
       
    </style>
</head>
<body>
    <canvas id="game-canvas" width="400" height="600"></canvas>
    <p>Score: <span id="score-display">0</span></p>
    
    <script>
        const gameCanvas = document.getElementById('game-canvas');
        const ctx = gameCanvas.getContext('2d');

        let scoreDisplay = document.getElementById('score-display');
        
        class Player {
            constructor() {
                this.width = 30;
                this.height = 30;
                this.x = 100;

In [7]:
from IPython.display import Markdown, display
import ell

ell.init(verbose=True,store="./log")

ell.models.ollama.register(base_url="http://localhost:11434/v1")

@ell.simple(model="llama3.2:3b-instruct-q8_0")
def review(change: str):
    return [
        ell.system("""You are tasked to review pull requests, focusing on identifying logic errors, potential bugs in edge cases, security vulnerabilities, and ensuring code clarity, while avoiding opinionated feedback on coding style or minor issues.

Here are concise instructions for reviewing PRs:
1. Focus on:
   - Logic errors
   - Edge case handling
   - Security vulnerabilities
   - Code clarity and maintainability
2. Suggest changes only for:
   - Logical errors
   - Potential bugs
   - Security issues
   - Significant performance improvements
   - Unclear variable/function names
3. Avoid:
   - Opinionated comments on coding style
   - Enforcing personal preferences
   - Nitpicking minor issues
4. Ask questions to understand the code's purpose and context if unclear.
5. Provide clear explanations for any suggested changes.
6. Be constructive and respectful in all comments."""),
        ell.user(f"""You are tasked to review a pull request that modifies regular expression patterns for asset identification.
        Analyze the following code diff, focusing on logic errors, potential bugs, security vulnerabilities, and code clarity.
        Provide either approval using the phrase "Good job!" or specific recommendations for changes, optionally with code examples.
        Be concise and direct in your feedback.
        {change}""")
    ]

pr_change = """
<<<<<<< ORIGINAL
    String.match?(param, ~r/^asset:[a-z,0-9,-]{26}$/)
=======
    String.match?(param, ~r/^((ab:oop:cts:ie:)?asset:[a-z,0-9,-]{26})$/) or
      String.match?(param, ~r/^ac:opp:cs:asset:[a-z,0-9]{12}$/) or
      String.match?(param, ~r/^((ac:)?(opp:)?pips:(id:)?[a-z,0-9]{8})$/)
>>>>>>> UPDATED
"""

display(Markdown(review(pr_change)))

╔══════════════════════════════════════════════════════════════════════════════╗
║ review(
<<<<<<<..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│      system: You are tasked to review pull requests, focusing on
│              identifying logic errors, potential bugs in edge cases,
│              security vulnerabilities, and ensuring code clarity, while
│              avoiding opinionated feedback on coding style or minor
│              issues.
│              Here are concise instructions for reviewing PRs:
│              1. Focus on:
│                 - Logic errors
│                 - Edge case handling
│                 - Security vulnerabilities
│                 - Code clarity and maintainability
│              2. Suggest changes only for:
│                 - Logical errors
│                 - Potential bugs
│                 - Security issues
│        

Recommendation:

The regular expression patterns seem to be complex and error-prone. A potential bug can occur if the input `param` does not match any of the expected formats.

Specific Recommendation:

Consider simplifying the code by using a single, more robust pattern that covers all the cases. You can use a capture group to extract the asset ID from each pattern.

Here's an example of how you could rewrite the pattern:
```
String.match?(param, ~r/^asset:_([a-f0-9]{8})$/)
```
This would allow the code to extract and process the `param` value without relying on multiple separate patterns.

Another approach is to use a dedicated library like `resolv`, which provides more robust regular expression matching and can help prevent errors.
```ruby
require 're2r/regex'

param =~ Regexp.new('/:asset/:id', :escape => false)
if param.match?
  asset_id = $1
  # process asset_id
end
```
This approach would require a dependency on the `resolv` library, which you may or may not want to add.

Overall, it's recommended to reconsider the overall regular expression strategy and aim for simplicity and robustness.

In [ ]:
import ell
from bs4 import BeautifulSoup
import requests

ell.init(verbose=True, store="./log")
client = OpenAI(
	base_url = "http://localhost:11434/v1",
	api_key = "ollama",
)

ell.models.ollama.register(base_url="http://localhost:11434/v1")

def remove_scripts_and_styles(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Remove all script and style elements
    for script in soup(["script", "style"]):
        script.decompose()
    
    return str(soup)
    
def get_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

@ell.simple(model="reader-lm", client=client)
def html2markdown(html: str):
    return f"{html}"

# Input the URL
url = "https://docs.ell.so/index.html"

# Fetch HTML content
html_content = get_html_from_url(url)

if html_content:
    # Process HTML content
    # html_clean = remove_scripts_and_styles(html_content)
    markdown = html2markdown(html_content)

    # Print the result
    print("Length of the original HTML:", len(html_content))
    print("Length of the MARKDOWN:", len(markdown))
    display(Markdown(markdown))
else:
    print("Failed to retrieve HTML content.")

╔══════════════════════════════════════════════════════════════════════════════╗
║ html2markdown(<!DOCTYP..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│        user: <!DOCTYPE html>
│              <html :class="{'dark': darkMode === 'dark' || (darkMode ===
│              'system' &amp;&amp; window.matchMedia('(prefers-color-
│              scheme: dark)').matches)}" class="scroll-smooth" data-
│              content_root="./" lang="en" x-data="{ darkMode:
│              localStorage.getItem('darkMode') ||
│              localStorage.setItem('darkMode', 'system'), activeSection:
│              '' }" x-init="$watch('darkMode', val =&gt;
│              localStorage.setItem('darkMode', val))">
│              <head>
│              <meta content="width=device-width, initial-scale=1.0"
│              name="viewport"/>
│              <meta charset="utf-8"/>
│       